In [7]:
from cvzone.HandTrackingModule import HandDetector
from cvzone.PoseModule import PoseDetector
import cv2
#Arduino
import cvzone.SerialModule
from serial import *
import serial
#serial = cvzone.SerialModule.SerialObject("/dev/cu.usbmodem1201",115200,1)

#fonction stabilité position;
def is_arduino_connected(port='"/dev/cu.usbmodem1401"'):
    try:
        arduino = serial.Serial(port, 115200)
        arduino.close()
        return print("Est connecté")
    except serial.SerialException:
        return print("Pas connecté")

is_arduino_connected();

def send_string_to_arduino(string, port='/dev/cu.usbmodem1401', baudrate=115200):

    arduino = serial.Serial(port, baudrate)
    arduino.write(string.encode())
    arduino.close()
send_string_to_arduino("G28");

def digit(nbr):
    sortie = str;
    if (int(nbr) <100):
        sortie =  "0" + str(int(nbr))
        return sortie
    else:
        sortie = str(int(nbr))
        return sortie

#Fonction Fermer ou Ouuvrir pINCE
def F(fingers):  # Deplacement F
    if fingers[1]==0 and fingers[2]==0 and (fingers[0]==0 or fingers[3]==0 or fingers[4] == 0):
        ##Voix a lancer
        
        return "010"
    else:
        ##Voix Ouvrir le pince
        return "073"

#   TestDATAS
aData = "090";
bData = "090";
cData = "090";
dData = "090";
eData = "090";
fData = "090";
fingers = [];
#Fonctions 


#Initialisation de la webcam + Detection 
cap = cv2.VideoCapture(0)
detector2 = PoseDetector()
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8, maxHands=1)





while True:
     

    # Get image frame
    success, imgnormal = cap.read();
    img = cv2.flip(imgnormal,1);
    # Find the hand and its landmarks
    hands, img = detector.findHands(img);  # with draw
    # hands = detector.findHands(img, draw=False)  # without draw


    # Get Buste
    imgPose = detector2.findPose(img)
    lmList, bboxInfo = detector2.findPosition(imgPose, bboxWithHands=False)
    if bboxInfo:
        center = bboxInfo["center"]
        cv2.circle(imgPose, center, 5, (255, 0, 255), cv2.FILLED)
        #Detection des angle
        angleCoude = detector2.findAngle(imgPose,16,14,12,draw=True);
        cData = angleCoude;
        angleEpaule = detector2.findAngle(imgPose,24,12,14,draw=True);
        anglePoignee = detector2.findAngle(imgPose,20,16,14,draw=True);
        dData = anglePoignee;
    
     



    if hands:
        # Hand 1
        hand1 = hands[0]
        lmList1 = hand1["lmList"]  # List of 21 Landmark points
        bbox1 = hand1["bbox"]  # Bounding box info x,y,w,h
        centerPoint1 = hand1['center']  # center of the hand cx,cy
        handType1 = hand1["type"]  # Handtype Left or Right  
        fingers1 = detector.fingersUp(hand1)
        fData = F(fingers1);
        print(fingers1)

       
            

            #mySerial.sendData(fingers1)
            # Find Distance between two Landmarks. Could be same hand or different hands
           # length, info, img = detector.findDistance(lmList1[8], lmList2[8], img)  # with draw
            # length, info = detector.findDistance(lmList1[8], lmList2[8])  # with draw
    else:
        #Voix Je sors du champs 
        print('je sors du champs')






    
    #print("before send gcode cData: " + str(cData));
    #print("before send gcode dData: " + str(dData));
    Gcode = "G01 " + "A" + digit(aData) +" " + "B" + digit(bData) +" " + "C"+ digit(cData)+ " " + "D" + digit(dData) + " " + "E" + digit(eData) + " " + "F" + digit(fData);
    print(Gcode)

    send_string_to_arduino(Gcode);
    #mySerial.sendData(Gcode)




    # Display
    cv2.imshow("Image", img)
    cv2.waitKey(1)
cap.release()
cv2.destroyAllWindows()

Pas connecté


SerialException: [Errno 16] could not open port /dev/cu.usbmodem1101: [Errno 16] Resource busy: '/dev/cu.usbmodem1101'